# <span style="color:blue"> MBA em Ciência de Dados</span>
# <span style="color:blue">Técnicas Avançadas para Captura e Tratamento de Dados</span>

## <span style="color:blue"> Matriz Documento $\times$ Palavras - Bag of Words</span>
    
## <span style="color:blue">Avaliação</span>

**Material Produzido por Luis Gustavo Nonato**<br>
**Cemeai - ICMC/USP São Carlos**
---

Os exercícios abaixo fazem uso da coleção de documentos presente no diretório `DocCol2` contido no arquivo <font style="font-family: monaco"> DocCol.zip</font>, o qual pode ser baixado do Moodle.

In [57]:
import glob
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

### Exercício 1)
Armazene os documentos disponíveis no diretório `DocCol2` em um dicionário onde a chave é o nome do arquivo e o valor é a string contida no arquivo. O documento contendo a string com o maior número de caracteres é:

a) gr7<br>
b) au2 <br>
c) ch5<br>
d) au8

In [18]:
# Lendo o documento como uma string
import glob  # glob é um pacote que permite varrer arquivos e 
             # diretorios utilizando sintaxe linux

files = glob.glob("DocCol2/*")

docs = {}
for fname in files:
    with open(fname,'r') as f:
        key = (fname.split('/')[-1]).replace('DocCol2\\','')
        docs[key] = f.read() 
        
print(docs.keys())

dict_keys(['au1', 'au10', 'au11', 'au12', 'au13', 'au14', 'au15', 'au2', 'au3', 'au4', 'au5', 'au6', 'au7', 'au8', 'au9', 'ch1', 'ch10', 'ch11', 'ch12', 'ch13', 'ch14', 'ch15', 'ch16', 'ch17', 'ch18', 'ch19', 'ch2', 'ch20', 'ch21', 'ch22', 'ch23', 'ch24', 'ch25', 'ch26', 'ch27', 'ch28', 'ch29', 'ch3', 'ch30', 'ch31', 'ch32', 'ch33', 'ch34', 'ch35', 'ch36', 'ch37', 'ch38', 'ch39', 'ch4', 'ch40', 'ch41', 'ch42', 'ch43', 'ch44', 'ch45', 'ch46', 'ch47', 'ch48', 'ch49', 'ch5', 'ch50', 'ch6', 'ch7', 'ch8', 'ch9', 'gr1', 'gr10', 'gr11', 'gr12', 'gr13', 'gr14', 'gr15', 'gr16', 'gr17', 'gr18', 'gr19', 'gr2', 'gr20', 'gr21', 'gr22', 'gr23', 'gr24', 'gr25', 'gr26', 'gr3', 'gr4', 'gr5', 'gr6', 'gr7', 'gr8', 'gr9'])


In [19]:
valor = 0
chave = ''

for key, values in docs.items():
    # print(len(values))
    if len(values) > valor:
        valor = len(values)
        chave = key

print(chave, valor)

au2 11672


## RESPOSTA:<br>
**LETRA B**

### Exercício 2)
Crie um dicionário chamado `docsXwords` onde as chaves são os nomes dos arquivos e os valores são as listas de palavras do documento correspondente. As palavras em cada uma das listas devem ser constituídas apenas por letras do alfabeto, estarem lexicamente normalizadas e conterem mais que 1 caracter. Qual o documento cuja lista de palavras resultante possui o **maior** número de palavras repetidas:

a) gr22<br>
b) ch30<br>
c) au2<br>
d) au8

In [42]:
docsXwords = {}

for key, value in docs.items():
    words = nltk.word_tokenize(value) #CRIA A LISTA DE PALAVRAS
    words = [w.lower() for w in words if w.isalpha() and len(w) != 1] #VERIFICA SE SÃO ALFABÉTICAS E COM MAIS DE 1 CARACTER
    words = [PorterStemmer().stem(w) for w in words] # FAZ A NORMALIZAÇÃO LÉXICA DA LISTA
    docsXwords[key] = words

In [43]:
# unificando todas as palavras em uma única lista
corpus = [palavras for sublista in list(docsXwords.values()) for palavras in sublista]

# removendo repeticoes
corpus = list(set(corpus))

df_dXp = pd.DataFrame(data=np.zeros((len(list(docs.keys())),len(corpus))),
                      index = list(docs.keys()), columns = corpus)

for key,value in docsXwords.items():
    dtemp = dict(Counter(value))
    df_dXp.loc[key,list(dtemp.keys())] = list(dtemp.values())

In [44]:
# for colum in df_dXp.columns():
#     print(df_dXp.sum())

print(df_dXp.sum(axis=1).sort_values(ascending=False).head(1))

au2    1678.0
dtype: float64


## RESPOSTA:<br>
**LETRA D**

### Exercício 3)
Utilizando as listas de palavras do dicionário `docsXwords`, quais as três palavras que mais aparecem na coleção de documentos:

a) the, is, of<br>
b) that, is, of<br>
c) the, of, to <br>
d) to, is, of

In [46]:
df_dXp.sum(axis=0).sort_values(ascending=False).head(3)

the    5014.0
of     2627.0
to     2611.0
dtype: float64

## RESPOSTA:<br>
**LETRA C**

### Exercício 4)
Qual o documento cuja lista de palavras possui o **menor** número de "stop words"? Quantas "stop words" aparecem neste documento:

a) gr5 com 47 "stop words"<br>
b) gr17 com 47 "stop words"<br>
c) gr5 com 37 "stop words"<br>
d) gr17 com 37 "stop words"

**Dica**: Crie um dicionário onde a chave é o nome do documento e o valor o número de stop words no documento.

In [54]:
doc_stop = {}

stop_words = stopwords.words('english')
for key, value in docsXwords.items():
    words = [w for w in value if w in stop_words]
    doc_stop[key] = len(words)


print(sorted(doc_stop.items(), key=lambda kv: kv[1], reverse=False)[0])


('gr5', 47)


## RESPOSTA:<br>
**LETRA A**

### Exercício 5) 
Utilize o dicionário `docsXwords` para construir
uma matriz Documentos $\times$ Palavras para a coleção de documentos do diretório `DocCol2`. Utilizando a distância cosseno, qual é o documento mais parecido com o documento 'ch7':

a) ch8<br>
b) ch16<br>
c) ch5<br>
d) au8

In [61]:
# encontrando o indice da linha de 'ch7' no DataFrame
ch7_id = np.argwhere(df_dXp.index.values=='ch7')[0][0]

X = df_dXp.values
X = StandardScaler().fit_transform(X)

# calculando o cosseno utilizando a formula
# cos(x,y) = np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))
cosch7 = np.apply_along_axis(lambda x: 
             np.dot(X[ch7_id],x)/(np.linalg.norm(ch7_id)*np.linalg.norm(x)),1,X)

# ordenando e pegando o maior valor (note que o elemento [-1] é o 
# próprio documento ch7)
sim_ch7_id = np.argsort(cosch7)[-2]
print('Documento mais parecido com ch7: ',df_dXp.index.values[sim_ch7_id])

Documento mais parecido com ch7:  ch16


## RESPOSTA:<br>
**LETRA B**